In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import pandas as pd

from lib.services import config

Using TensorFlow backend.


In [2]:
train = pd.read_csv(config['dataset']['path']['train'])
test = pd.read_csv(config['dataset']['path']['test'])

train_features = train[config['dataset']['feature']].fillna("_na_").values
test_features = test[config['dataset']['feature']].fillna("_na_").values

In [3]:
tokenizer = Tokenizer(num_words=config['word_embeding']['rows_count'])
tokenizer.fit_on_texts(list(train_features))

tokenized_train = tokenizer.texts_to_sequences(train_features)
X_train = pad_sequences(
    tokenized_train, 
    maxlen=config['word_embeding']['max_words_count']
)

tokenized_test = tokenizer.texts_to_sequences(test_features)
X_test = pad_sequences(
    tokenized_test, 
    maxlen=config['word_embeding']['max_words_count']
)

In [4]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')

embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(config['word_embeding']['path']))

FileNotFoundError: [Errno 2] No such file or directory: 'word-embeding/glove.6B.300d.txt'

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

In [ ]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=0.1);

In [ ]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv(f'{path}{comp}sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission.csv', index=False)